In [1]:
import numpy as np
import pandas 
from sklearn.feature_extraction.text import TfidfVectorizer


In [31]:
#1. Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv 
# (либо его заархивированную версию salary-train.zip).
train = pandas.read_csv('salary-train.csv')# , header=None)
train_test = pandas.read_csv('salary-test-mini.csv')# , header=None)
train[:2]

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000


In [47]:
train_test[:2]

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,we currently have a vacancy for an hr project ...,milton keynes,contract,NaN
1,a web developer opportunity has arisen with an...,manchester,permanent,NaN


In [32]:
# 2. Проведите предобработку: Приведите тексты к нижнему регистру (text.lower()).
train['FullDescription'] = train['FullDescription'].str.lower()
train['LocationNormalized'] = train['LocationNormalized'].str.lower()
train_test['FullDescription'] = train_test['FullDescription'].str.lower()
train_test['LocationNormalized'] = train_test['LocationNormalized'].str.lower()

In [14]:
train[:2]

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,london,permanent,33000
1,an ideal opportunity for an individual that ha...,london,permanent,50000


In [33]:
# Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова. 
# Для такой замены в строке text подходит следующий вызов: re.sub('[^a-zA-Z0-9]', ' ', text). 
# Также можно воспользоваться методом replace у DataFrame, чтобы сразу преобразовать все тексты:
train['FullDescription'] = train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
train_test['FullDescription'] = train_test['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

In [16]:
train[:2]

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,london,permanent,33000
1,an ideal opportunity for an individual that ha...,london,permanent,50000


In [46]:
train_test[:2]

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,we currently have a vacancy for an hr project ...,milton keynes,contract,NaN
1,a web developer opportunity has arisen with an...,manchester,permanent,NaN


In [34]:
#Примените TfidfVectorizer для преобразования текстов в векторы признаков. 
# Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).
vectorizer=TfidfVectorizer(min_df=5)

In [37]:
train_sel=vectorizer.fit_transform(train['FullDescription'])

In [50]:
train_sel

<60000x22861 sparse matrix of type '<class 'numpy.float64'>'
	with 8365759 stored elements in Compressed Sparse Row format>

In [39]:
#train_test_sel=vectorizer.fit(train_test['FullDescription'])

In [40]:
# Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'. 
# Код для этого был приведен выше.
train['LocationNormalized'].fillna('nan', inplace=True)
train['ContractTime'].fillna('nan', inplace=True)
train_test['LocationNormalized'].fillna('nan', inplace=True)
train_test['ContractTime'].fillna('nan', inplace=True)

In [43]:
# Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.
from sklearn.feature_extraction import DictVectorizer
enc = DictVectorizer()
X_train_categ = enc.fit_transform(train[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_train_test_categ = enc.transform(train_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [44]:
X_train_categ

<60000x1766 sparse matrix of type '<class 'numpy.float64'>'
	with 120000 stored elements in Compressed Sparse Row format>

In [45]:
X_train_test_categ

<2x1766 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [48]:
# Объедините все полученные признаки в одну матрицу "объекты-признаки". Обратите внимание, что матрицы 
# для текстов и категориальных признаков являются разреженными. Для объединения их столбцов нужно воспользоваться 
# функцией scipy.sparse.hstack.
from scipy.sparse import hstack

In [51]:
#hstack([A,B]).toarray()
hstack([train_sel,X_train_categ]).toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])